In [1]:
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()
# 2021.10.08 데이터 추출 완료 - 31140 데이터가 대부분 없음, InterfaceError: Python type ResultSet cannot be converted > updowncd 값 2에 의한 에러
cursor = mydb.cursor(prepared=True)
citycode = '36420'

query1 = ("select citycode from public_busctycodelist where citycode >= {}".format(citycode))

cursor.execute(query1)

location_list1 = []

for a in cursor:
    location_list1.append(a)

now = datetime.now()
nowDate = now.strftime('%Y%m%d')

b_cnt = 0
for z in location_list1:
    if b_cnt == 1:
        break
        
    location = z
    query2 = ("select distinct routeid from public_busroute_info where citycode = '{}'".format(location[0]))
    cursor.execute(query2)

    location_list2 = []

    for b in cursor:
        location_list2.append(b)

    cnt = 1
    for i in location_list2:
        routeId = i

        characters = "(',')"

        join_data = ''.join( x for x in routeId if x not in characters)
        print("조회날짜 : {} | location : {} | cnt : {} | start {} public_busroute_info".format(nowDate,location[0],cnt,join_data))

        url = 'http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteAcctoThrghSttnList'
        queryParams = '?' + urlencode({
            quote_plus('serviceKey') : public_serviceKey(),
            quote_plus('cityCode') : location[0],
            quote_plus('pageNo') : '1',
            quote_plus('numOfRows') : '10000',
            quote_plus('routeId') : join_data
        })

        response = requests.get(url + queryParams).text.encode('utf-8')
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        rows = xmlobj.findAll('item')
        resultCode = xmlobj.findAll('resultCode')
        if resultCode[0].text == '99':
            print("location {} | {} 번째부터 조회수 제한이 걸렸습니다.".format(location[0],join_data))
            mydb.close()
            b_cnt += 1
            break
        if rows == []:
            print("location {} | {} 데이터가 없습니다.".format(location[0],join_data))
            cnt += 1
            continue
        cnt += 1
        for j in rows:
            item = j
            lat = item.findAll('gpslati')                
            lng = item.findAll('gpslong')
            nodeid = item.findAll('nodeid')
            nodenm = item.findAll('nodenm')
            nodeno = item.findAll('nodeno')
            if nodeno == []:
                nodeno = None
            else:
                nodeno = nodeno[0].text
            nodeord = item.findAll('nodeord')
            routeid = item.findAll('routeid')  
            updowncd = item.findAll('updowncd')
            
            if updowncd == []:
                updowncd = None
            elif updowncd[0].text == '0':
                updowncd = '상행'
            elif updowncd[0].text == '1':
                updowncd = '하행'
            elif updowncd[0].text == '2':
                updowncd = '2'

            sql = "INSERT INTO public_busroute_detail (lat, lng, nodeid, nodenm, nodeno, nodeord, routeid, updowncd, citycode) VALUES (?,?,?,?,?,?,?,?,?)"
            val = (lat[0].text, lng[0].text, nodeid[0].text, nodenm[0].text, nodeno, nodeord[0].text, routeid[0].text, updowncd, location[0])

            cursor.execute(sql, val)
            mydb.commit()

조회날짜 : 20211009 | location : 36030 | cnt : 1 | start SCB326000244 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 2 | start SCB326000001 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 3 | start SCB326000002 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 4 | start SCB326000003 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 5 | start SCB326000004 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 6 | start SCB326000005 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 7 | start SCB326000009 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 8 | start SCB326000010 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 9 | start SCB326000013 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 10 | start SCB326000014 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 11 | start SCB326000015 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt 

조회날짜 : 20211009 | location : 36030 | cnt : 95 | start SCB326000056 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 96 | start SCB326000058 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 97 | start SCB326000059 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 98 | start SCB326000060 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 99 | start SCB326000061 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 100 | start SCB326000062 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 101 | start SCB326000063 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 102 | start SCB326000064 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 103 | start SCB326000065 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 104 | start SCB326000066 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 105 | start SCB326000068 public_busroute_info
조회날짜 : 20211009 | location

조회날짜 : 20211009 | location : 36030 | cnt : 188 | start SCB326000226 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 189 | start SCB326000155 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 190 | start SCB326000156 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 191 | start SCB326000193 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 192 | start SCB326000194 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 193 | start SCB326000219 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 194 | start SCB326000220 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 195 | start SCB326000176 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 196 | start SCB326000177 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 197 | start SCB326000196 public_busroute_info
조회날짜 : 20211009 | location : 36030 | cnt : 198 | start SCB326000197 public_busroute_info
조회날짜 : 20211009 | loc

조회날짜 : 20211009 | location : 36060 | cnt : 85 | start KYB328000024 public_busroute_info
조회날짜 : 20211009 | location : 36330 | cnt : 1 | start TSB331000001 public_busroute_info
조회날짜 : 20211009 | location : 36330 | cnt : 2 | start TSB331000005 public_busroute_info
조회날짜 : 20211009 | location : 36330 | cnt : 3 | start TSB331000006 public_busroute_info
조회날짜 : 20211009 | location : 36330 | cnt : 4 | start TSB331000010 public_busroute_info
조회날짜 : 20211009 | location : 36330 | cnt : 5 | start TSB331000020 public_busroute_info
조회날짜 : 20211009 | location : 36330 | cnt : 6 | start TSB331000021 public_busroute_info
조회날짜 : 20211009 | location : 36330 | cnt : 7 | start TSB331000029 public_busroute_info
조회날짜 : 20211009 | location : 36330 | cnt : 8 | start TSB331000030 public_busroute_info
조회날짜 : 20211009 | location : 36330 | cnt : 9 | start TSB331000031 public_busroute_info
조회날짜 : 20211009 | location : 36330 | cnt : 10 | start TSB331000032 public_busroute_info
조회날짜 : 20211009 | location : 36330 | cnt 

조회날짜 : 20211009 | location : 36350 | cnt : 26 | start TSB332000317 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 27 | start TSB332000318 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 28 | start TSB332000319 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 29 | start TSB332000321 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 30 | start TSB332000322 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 31 | start TSB332000324 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 32 | start TSB332000325 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 33 | start TSB332000327 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 34 | start TSB332000339 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 35 | start TSB332000346 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 36 | start TSB332000357 public_busroute_info
조회날짜 : 20211009 | location : 363

...

조회날짜 : 20211009 | location : 36350 | cnt : 213 | start TSB332000353 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 214 | start TSB332000362 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 215 | start TSB332000364 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 216 | start TSB332000365 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 217 | start TSB332000366 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 218 | start TSB332000369 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 219 | start TSB332000370 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 220 | start TSB332000372 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 221 | start TSB332000373 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 222 | start TSB332000374 public_busroute_info
조회날짜 : 20211009 | location : 36350 | cnt : 223 | start TSB332000375 public_busroute_info
조회날짜 : 20211009 | loc

조회날짜 : 20211009 | location : 36400 | cnt : 50 | start TSB337000103 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 51 | start TSB337000106 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 52 | start TSB337000109 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 53 | start TSB337000112 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 54 | start TSB337000113 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 55 | start TSB337000114 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 56 | start TSB337000115 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 57 | start TSB337000116 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 58 | start TSB337000117 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 59 | start TSB337000118 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 60 | start TSB337000119 public_busroute_info
조회날짜 : 20211009 | location : 364

조회날짜 : 20211009 | location : 36400 | cnt : 143 | start TSB337000125 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 144 | start TSB337000126 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 145 | start TSB337000127 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 146 | start TSB337000129 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 147 | start TSB337000132 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 148 | start TSB337000141 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 149 | start TSB337000146 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 150 | start TSB337000161 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 151 | start TSB337000162 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 152 | start TSB337000163 public_busroute_info
조회날짜 : 20211009 | location : 36400 | cnt : 153 | start TSB337000165 public_busroute_info
조회날짜 : 20211009 | loc

조회날짜 : 20211009 | location : 36410 | cnt : 5 | start MPB338000116 public_busroute_info
조회날짜 : 20211009 | location : 36410 | cnt : 6 | start MPB338000117 public_busroute_info
조회날짜 : 20211009 | location : 36410 | cnt : 7 | start MPB338000118 public_busroute_info
조회날짜 : 20211009 | location : 36410 | cnt : 8 | start MPB338000119 public_busroute_info
조회날짜 : 20211009 | location : 36410 | cnt : 9 | start MPB338000120 public_busroute_info
조회날짜 : 20211009 | location : 36410 | cnt : 10 | start MPB338000002 public_busroute_info
조회날짜 : 20211009 | location : 36410 | cnt : 11 | start MPB338000003 public_busroute_info
조회날짜 : 20211009 | location : 36410 | cnt : 12 | start MPB338000004 public_busroute_info
조회날짜 : 20211009 | location : 36410 | cnt : 13 | start MPB338000005 public_busroute_info
조회날짜 : 20211009 | location : 36410 | cnt : 14 | start MPB338000008 public_busroute_info
조회날짜 : 20211009 | location : 36410 | cnt : 15 | start MPB338000009 public_busroute_info
조회날짜 : 20211009 | location : 36410 | 

조회날짜 : 20211009 | location : 36420 | cnt : 22 | start MPB339000080 public_busroute_info
조회날짜 : 20211009 | location : 36420 | cnt : 23 | start MPB339000082 public_busroute_info
조회날짜 : 20211009 | location : 36420 | cnt : 24 | start MPB339000089 public_busroute_info
조회날짜 : 20211009 | location : 36420 | cnt : 25 | start MPB339000102 public_busroute_info
조회날짜 : 20211009 | location : 36420 | cnt : 26 | start MPB339000118 public_busroute_info
조회날짜 : 20211009 | location : 36420 | cnt : 27 | start MPB339000120 public_busroute_info
조회날짜 : 20211009 | location : 36420 | cnt : 28 | start MPB339000002 public_busroute_info
조회날짜 : 20211009 | location : 36420 | cnt : 29 | start MPB339000006 public_busroute_info
조회날짜 : 20211009 | location : 36420 | cnt : 30 | start MPB339000007 public_busroute_info
조회날짜 : 20211009 | location : 36420 | cnt : 31 | start MPB339000011 public_busroute_info
조회날짜 : 20211009 | location : 36420 | cnt : 32 | start MPB339000012 public_busroute_info
조회날짜 : 20211009 | location : 364